ЛДА на всем датасете

In [ ]:
#pip install --upgrade ipykernel

In [1]:
!pip install pyLDAvis

You should consider upgrading via the 'c:\users\philipp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\cupy\__init__.py:103: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  from numpy import bool8  # NOQA
c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\cupy\__init__.py:103: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  from numpy import bool8  # NOQA
c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\cupy\__init__.py:114: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  from numpy import int0  # NOQA
c:\user

In [3]:
train = pd.read_csv('train2.csv', sep='\t', encoding='ISO-8859-1')
test = pd.read_csv('test2.csv',sep='\t',encoding='ISO-8859-1')

In [4]:
train.drop(columns = ['Unnamed: 0','body'], inplace = True)
test.drop(columns = ['Unnamed: 0','body'], inplace = True)

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [6]:
def bigrams(words, bi_min=5, tri_min=5):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [7]:
def get_corpus(df):

    words = list(sent_to_words(df.body_new))
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(body_new) for body_new in bigram]
    return corpus, id2word, bigram

In [8]:
train_corpus, train_id2word, bigram_train = get_corpus(train)

In [9]:
TOPICS = 3

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=TOPICS,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train3.model')

In [10]:
train_vecs = []
for i in range(len(train)):
    top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(TOPICS)]
    train_vecs.append(topic_vec)

In [11]:
X3 = np.array(train_vecs)

In [12]:
TOPICS = 20

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=TOPICS,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train20.model')

In [13]:
train_vecs = []
for i in range(len(train)):
    top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(TOPICS)]
    train_vecs.append(topic_vec)

In [14]:
X20 = np.array(train_vecs)

In [19]:
cols = train.drop(columns = ['Y', 'body_new']).columns

In [16]:
cols = train.drop(columns = ['Unnamed: 0','body', 'Y', 'body_new']).columns

KeyError: "['Unnamed: 0', 'body'] not found in axis"

In [20]:
cols = np.array(cols)

In [21]:
train_vecs = []

for i in range(len(train)):
  topic_vec = []
  for c in cols:
    topic_vec.extend([train.iloc[i][c]])   
  train_vecs.append(topic_vec)

In [22]:
features = np.array(train_vecs)
X_train = np.concatenate((X3,X20,features), axis=1)
y_train = np.array(train.Y)

In [23]:
def get_bigram(df):
    words = list(sent_to_words(df.body_new))
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    return bigram

In [24]:
bigram_test = get_bigram(test)

In [25]:
lda_train3 = gensim.models.ldamulticore.LdaMulticore.load('lda_train3.model')
lda_train20 = gensim.models.ldamulticore.LdaMulticore.load('lda_train20.model')

In [26]:
test_corpus = [train_id2word.doc2bow(body_new) for body_new in bigram_test]

In [27]:
test_vecs3 = []
for i in range(len(test)):
    top_topics = lda_train3.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
    test_vecs3.append(topic_vec)

X3 = np.array(test_vecs3)

test_vecs20 = []
for i in range(len(test)):
    top_topics = lda_train20.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    test_vecs20.append(topic_vec)

X20 = np.array(test_vecs20)

In [28]:
test_vecs = []

for i in range(len(test)):
  topic_vec = []
  for c in cols:
    topic_vec.extend([test.iloc[i][c]])   
  test_vecs.append(topic_vec)

In [29]:
features = np.array(test_vecs)
X_test = np.concatenate((X3,X20,features), axis=1)
y_test = np.array(test.Y)

In [32]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def metrics(y_test, y_pred, y_pred_proba):
  
  conf = confusion_matrix(y_test, y_pred)
  print(conf)
  f1_ma = round(f1_score(y_test, y_pred, average='macro'), 4)
  f1_mi = round(f1_score(y_test, y_pred, average='micro'), 4)
  f1_we = round(f1_score(y_test, y_pred, average='weighted'), 4)
  b_acc = round(balanced_accuracy_score(y_test, y_pred), 4)
  neg_ll = round(- log_loss(y_test, y_pred_proba), 4)
  roc_auc_ovr = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovr'), 4)
  roc_auc_ovo = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovo'), 4)

  print('f1_score_macro:', f1_ma)
  print('f1_score_micro:', f1_mi)
  print('f1_score_weighted:', f1_we)
  print('balanced_accuracy_score:', b_acc)
  print('neg_log_loss:', neg_ll)
  print('roc_auc_score ovr:', roc_auc_ovr)
  print('roc_auc_score ovo:', roc_auc_ovo)

  return pd.DataFrame({'features': ['+LDA'],'f1_score_macro': [f1_ma], 'f1_score_micro': [f1_mi],'f1_score_weighted': [f1_we], 'balanced_accuracy_score': [b_acc], 'neg_log_loss': [neg_ll],'roc_auc_score ovr': [roc_auc_ovr], 'roc_auc_score ovo': [roc_auc_ovo]})


In [33]:
pipel = Pipeline(steps = [('scaler', StandardScaler()), ('model',LogisticRegression(max_iter=10000,class_weight = 'balanced'))])

pipel.fit(X_train, y_train)

y_pred = pipel.predict(X_test)
y_pred_proba = pipel.predict_proba(X_test)

res = metrics(y_test, y_pred, y_pred_proba)

[[45956  5008  2196]
 [  171  1403   418]
 [ 2681  4967  5784]]
f1_score_macro: 0.5471
f1_score_micro: 0.7749
f1_score_weighted: 0.8085
balanced_accuracy_score: 0.6665
neg_log_loss: -0.698
roc_auc_score ovr: 0.8767
roc_auc_score ovo: 0.8729


In [ ]:
with open('drive/MyDrive/Diplom/results.csv') as f:
    results = pd.read_csv(f)

In [34]:
results = pd.read_csv('results.csv', sep='\t', encoding='ISO-8859-1')


In [36]:
results = pd.concat([results, res])
# results.drop(columns = ['Unnamed: 0'], inplace = True)

In [37]:
columns = np.concatenate((np.array(['a3','b3','c3','d20','e20','f20','g20','h20','i20','j20','k20','l20','m20','n20','o20','p20','q20','r20','s20','t20','u20','v20','w20']),cols), axis=0)

In [38]:
test = pd.DataFrame(X_test, columns = columns)
test['Y'] = y_test

In [39]:
train = pd.DataFrame(X_train, columns = columns)
train['Y'] = y_train

In [40]:
train.to_csv('train3.csv', sep='\t')
test.to_csv('test3.csv', sep='\t')
results.to_csv('results.csv')